In [7]:
import openai

client = openai.OpenAI()
messages = []

In [ ]:
def get_weather(city):
    return "33 deegress celcius."

In [ ]:
# 모델에게 함수들의 설명을 전달하기 위함.

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of",
                    }
                },
                "required": ["city"],
            },
        }
    }
]

In [ ]:
def call_ai():
    response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=TOOLS
        )
    # 모델은 도구를 사용하고 싶을때, message.content 안에 write 하지 않는다.
    print(response)
    message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": message})
    print(f"Assistant: {message}")

while True:
    message = input("Send a message to the LLM")
    if message == "quit" or message == 'q':
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()
        

User: My name is luke
Assistant: Nice to meet you, Luke! How can I assist you today?
User: What is my name ?
Assistant: Your name is Luke.
User: Im from korea
Assistant: That's great, Luke! Korea has a rich culture and history. Are you from South Korea or North Korea?
User: What was the first question i asked you and what is the closest Island country to where I was born ?
Assistant: The first question you asked me was, "What is my name?" 

As for the closest island country to Korea, it's Japan. Japan is located to the east of the Korean Peninsula, across the Korea Strait. If you were born in Korea, Japan is the nearest island nation.
User: 
Assistant: It looks like your message might have been cut off. How can I assist you further?


KeyboardInterrupt: Interrupted by user